In [5]:
import robotic as ry
import matplotlib.pyplot as plt
import numpy as np
import time
pclColor = [0,0,255]
max_trials = 10
max_vel = 100
max_acc = max_vel

C = ry.Config()
C.addFile('/home/vboxuser/Desktop/CS449/Migi/hw3/GFiles-HW3/environment.g')

# C.addFile('./environment.g')
# C.addFile(ry.raiPath('environment.g'))
# qHome = C.getJointState()
# print(qHome)
C.view()
print(ry.raiPath('environment.g'))

/home/vboxuser/Desktop/CS449/rai_venv/lib/python3.8/site-packages/robotic/rai-robotModels/environment.g


# 1.1 Perception Module

In [6]:
numOfCameras = 1

### Get point clouds for each camera

In [7]:
cam = [None] * numOfCameras
rgb = [None] * numOfCameras
depth = [None] * numOfCameras
pclOrig = [None] * numOfCameras
pcl = [None] * numOfCameras
for i in range(numOfCameras):
  cam[i] = ry.CameraView(C)
  cam[i].setCamera('camera'+str(i+1))
  rgb[i], depth[i] = cam[i].computeImageAndDepth(C)
  pclOrig[i] = ry.depthImage2PointCloud(depth[i], cam[i].getFxycxy())
  pcl[i] = ry.depthImage2PointCloud(depth[i], cam[i].getFxycxy())
  print(pcl[i].shape)
  
pclFramesOrig = [None] * numOfCameras

for i in range(numOfCameras):
  pclFramesOrig[i] = C.addFrame('pclOrig'+str(i+1))
  pclFramesOrig[i].setPointCloud(pcl[i], [ 0, 255, 0])

C.view()
  
# print(rgb.shape, depth.shape, pcl1.shape)
# print(C.view_fxycxy())

(360, 640, 3)


0

### Now normalise the points relative to world frame

In [ ]:
for i in range(numOfCameras):
  curCam = C.getFrame("camera"+str(i+1))
  curCamX = curCam.getRelativePosition()[0]
  curCamY = curCam.getRelativePosition()[1]
  curCamZ = curCam.getRelativePosition()[2]
  
  curCamPos = curCam.getRelativePosition()
  curCamRotMat = curCam.getRotationMatrix()
  curCamRotMatTransposed = np.transpose(curCamRotMat)
  
  t_wc = -np.dot(curCamRotMatTransposed, curCamPos)

  T_wc = np.eye(4)
  T_wc[:3, :3] = curCamRotMatTransposed
  T_wc[:3, 3] = t_wc
  


  
    
  for h in range(pclOrig[i].shape[0]):    # Iterate over height
    for w in range(pclOrig[i].shape[1]):  # Iterate over width
      point = pclOrig[i][h, w]            # Get the 3D point [x, y, z]
      eachX, eachY, eachZ = point     # Unpack point coordinates
      
      # now make these positions relative to the world and hence bounding box
      eachX = eachX - curCamX
      eachY = eachY - curCamY
      eachZ = eachZ - curCamZ
      
      pcl[i][h, w] = [eachX, eachY, eachZ]
      
      

pclFrames = [None] * numOfCameras

for i in range(numOfCameras):
  # pclFrames[i] = C.addFrame('pcl'+str(i+1), 'camera'+str(i+1))
  pclFrames[i] = C.addFrame('pcl'+str(i+1), 'world')
  # pclFrames[i].setPose('t(-0.5 -0.1 1.08) d(-120 1 0 0) d(45 0 1 0)')
  pclFrames[i].setPointCloud(pcl[i], [ 255, 0, 0])


# C.view()

# # print(C.getFrameNames())

Rot [[ 0.70710678  0.          0.70710678]
 [-0.61237244 -0.5         0.61237244]
 [ 0.35355339 -0.8660254  -0.35355339]]
Transpose of Rot [[ 0.70710678 -0.61237244  0.35355339]
 [ 0.         -0.5        -0.8660254 ]
 [ 0.70710678  0.61237244 -0.35355339]]

-- WARNING:kin.cpp:addFrame:193(-1) frame already exists! returning existing without modifications!


### Alternatively, Normalise using frames

In [421]:
# pclFramesOrig[0].setPose('t(-0.5 -0.1 1.08) d(-120 1 0 0) d(45 0 1 0)')
# pclFramesOrig[1].setPose('t(0.5 -0.1 1.08) d(-120 1 0 0) d(-45 0 1 0)')
# pclFramesOrig[2].setPose('t(0.5 1.0 1.08) d(180 0 1 0) d(60 1 0 0) d(45 0 1 0)')
# pclFramesOrig[3].setPose('t(-0.5 1.0 1.08) d(180 0 1 0) d(60 1 0 0) d(-45 0 1 0)')
C.view()

0

### Now we extract the ones inside the bounding box

In [398]:
# bounding_box = C.addFrame("bounding_box", "bin")
# bounding_box.setShape(ry.ST.box, size=[0.6, 0.35, 0.1])  # A box of size 0.5x0.5x0.5
# bounding_box.setRelativePosition([0., -0.05, 0.08])  # Position the bounding box





centre = [0, 0.4, 0.68]
# offsets = [0.05, 0.35, 0.3]
offsets = [.1, .1, .1]

bounding_box_min_x = centre[0] - offsets[0]
bounding_box_max_x = centre[0] + offsets[0]

bounding_box_min_y = centre[1] - offsets[1]
bounding_box_max_y = centre[1] + offsets[1]

bounding_box_min_z = centre[2] - offsets[2]
bounding_box_max_z = centre[2] + offsets[2]

# print(bounding_box.getPosition())

for i in range(numOfCameras):
  for h in range(pcl[i].shape[0]):  # Iterate over height
    for w in range(pcl[i].shape[1]):  # Iterate over width
      point = pcl[i][h, w]  # Get the 3D point [x, y, z]
      eachX, eachY, eachZ = point  # Unpack point coordinates
      
    # Check if the point is within the bounding box
    if not (bounding_box_min_x < eachX < bounding_box_max_x and
      bounding_box_min_y < eachY < bounding_box_max_y and
      bounding_box_min_z < eachZ < bounding_box_max_z):
        pcl[i][h, w] = [0, 0, 0] 


C.view()

0

In [399]:
# finalpcl = pcl[0] + pcl[1] + pcl[2] + pcl[3]
# finalpclFrame = C.addFrame('finalpclFrame', 'world')
# finalpclFrame.setPointCloud(finalpcl, [ 255, 0, 0])
# C.view()

# print("pcl1: ", pcl[0].shape)
# print("pcl2: ", pcl[1].shape)

# extract the point cloud inside bounding box
# boundingBox = [0.5, 0.5, 0.5, 0.6, 0.6, 0.6]

# print positions of pcl points
# for point in pcl[0]:
  # print(point.getPosition())
